# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
import pickle

from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import GridSearchCV

In [2]:
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("CategorizedResponse", engine)
X = df["message"]
y = df.drop(["id", "message", "original", "genre"], axis = "columns")
y.head()
#df.head()
#X.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    """ Tokenize """
    
    # Normalize text
    # Replace non-alphanumeric characters with space, convert to lower and remove trailing spaces 
    text = re.sub("[^a-zA-Z09]", " ", text).lower().strip()
    # To be extra-clean remove double spaces
    text = re.sub("\s{2,}", " ", text)
    
    # After normalization, tokenize text into tokens, remove stop words and lemmatize
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words("english")]
    
    return tokens

print(tokenize(X[0]))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def createPipeline(classifier):
    """ Function to create the pipeline with tokenization plus classifier wrapped in multi output """
    return Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(classifier))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
# Note: Regarding some problems I had, in the future I should look into "stratify = y" option and StratifiedShuffleSplit
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [7]:
print(len(y_train))

19661


In [8]:
# Note: Old code before I switched to pipeline, kept here for future debugging if something doesn't work

#vect = CountVectorizer(tokenizer = tokenize)
#tfidf = TfidfTransformer()
#moc = MultiOutputClassifier(RandomForestClassifier())
#X_train_counts = vect.fit_transform(X_train)
#X_train_tfidf = tfidf.fit_transform(X_train_counts)

# Note: I had multiple problems here because I used SVC and there were exclusively 0 values in child_alone. 
# Fixed by removing the obsolete target child_alone and using a different classifier.
#moc.fit(X_train_tfidf, y_train)

# Vectorize test input data
# Note: Got errors here because I used fit_transform instead of transform and because I created nwwe vect and tfidf instances.
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)

# Note: When using SVC as clsf. I got all 0 predictions for the labels. Probably need to tune SVC. Fixed by using differnt clsf.
#y_pred = moc.predict(X_test_tfidf)

In [9]:
# Fit the classification model
pipeline = createPipeline(RandomForestClassifier())
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

print (y_pred[1009]) # sample

[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0]


In [11]:
# Note: I had problems here becuase the data wasn't clean. Fixed by removing "2" from target "related" in ETL.
print(classification_report(y_test, y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      5001
               request       0.79      0.42      0.55      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.77      0.60      0.68      2700
          medical_help       0.62      0.11      0.18       532
      medical_products       0.70      0.07      0.12       345
     search_and_rescue       0.75      0.04      0.07       165
              security       0.40      0.02      0.03       127
              military       0.62      0.09      0.16       197
           child_alone       0.00      0.00      0.00         0
                 water       0.83      0.28      0.42       408
                  food       0.83      0.34      0.49       723
               shelter       0.80      0.35      0.49       590
              clothing       0.81      0.14      0.23        95
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Create a smaller training set to run gridsearch on
cv_size_divisor = 2.5

X_cv_reduced = X_train.copy(deep = True)[:int(len(X_train) / cv_size_divisor)]
y_cv_reduced = y_train.copy(deep = True)[:int(len(y_train) / cv_size_divisor)]

#print(len(X_cv_reduced))
#print(len(y_cv_reduced))
#X_cv_reduced.head()
#y_cv_reduced.head()

In [13]:
#RandomForestClassifier().get_params().keys()

In [ ]:
# Run gridsearch
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0),
    'clf__estimator__n_estimators': [10, 30] 
}

cv = GridSearchCV(estimator = pipeline, param_grid = parameters, scoring='f1_macro', cv = None, n_jobs = -1, verbose = 10)
cv.fit(X_cv_reduced, y_cv_reduced)

y_pred_cv = cv.predict(X_test)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5, vect__ngram_range=(1, 1) 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.2102804340872405, total=  44.0s
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=10, vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.22339417089733551, total=  44.0s
[CV] clf__estimator__n_estimators=10, vect__max_df=0.5, vect__ngram_range=(1, 1) 


...

[CV]  clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.218240052323343, total= 1.0min
[CV] clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 2) 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.18511504942243764, total= 1.3min
[CV] clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 2) 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[CV]  clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.18907456136018874, total= 1.3min
[CV] clf__estimator__n_estimators=30, vect__max_df=1.0, vect__ngram_range=(1, 2) 


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test, y_pred_cv, target_names = y.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:
# 1 Try a different classifier: LinearSVC

# SVC will not accept singular class, remove offending column
y_test.drop(["child_alone"], axis = "columns", inplace = True)
y_train.drop(["child_alone"], axis = "columns", inplace = True)

pipeline = createPipeline(LinearSVC())

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("LinearSVC")
print(classification_report(y_test, y_pred, target_names = y_train.columns))

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


LinearSVC
                        precision    recall  f1-score   support

               related       0.86      0.91      0.89      5001
               request       0.76      0.59      0.67      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.72      0.67      0.70      2700
          medical_help       0.59      0.30      0.40       532
      medical_products       0.64      0.30      0.41       345
     search_and_rescue       0.66      0.15      0.25       165
              security       0.57      0.03      0.06       127
              military       0.50      0.27      0.36       197
                 water       0.77      0.60      0.67       408
                  food       0.82      0.69      0.75       723
               shelter       0.75      0.55      0.64       590
              clothing       0.75      0.44      0.56        95
                 money       0.53      0.22      0.32       138
        missing_people       

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0)
}

cv = GridSearchCV(estimator = pipeline, param_grid = parameters, scoring='f1_macro', cv = None, n_jobs = -1, verbose = 10)
cv.fit(X_cv_reduced, y_cv_reduced)

y_pred_cv = cv.predict(X_test)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1) ......................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.3483433719243196, total=  40.6s
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1) ......................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  vect__max_df=0.5, vect__ngram_range=(1, 1), score=0.36111489354290727, total=  40.6s
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1) ......................


...

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.3608932124169923, total=  41.8s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2) ......................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.34353783534064103, total=  41.7s


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 13.6min finished


In [12]:
print(classification_report(y_test, y_pred_cv, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.86      0.91      0.88      5001
               request       0.75      0.58      0.65      1093
                 offer       0.00      0.00      0.00        32
           aid_related       0.71      0.68      0.69      2700
          medical_help       0.57      0.27      0.37       532
      medical_products       0.66      0.28      0.39       345
     search_and_rescue       0.74      0.17      0.28       165
              security       0.25      0.01      0.02       127
              military       0.54      0.18      0.27       197
           child_alone       0.75      0.61      0.67       408
                 water       0.82      0.66      0.73       723
                  food       0.73      0.52      0.61       590
               shelter       0.76      0.40      0.52        95
              clothing       0.53      0.15      0.24       138
                 money       0.73      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
# 2 Try another classifier: AdaBoost. 
# Also, since the dataset is imbalanced, remove columns with insufficient data. 

labelsToDrop = ['offer', 'security', 'tools', 'hospitals', 'shops', 'aid_centers']
y_TrainDroppedLabels = y_train.copy()
y_TrainDroppedLabels.drop(labelsToDrop, axis = "columns", inplace = True)
y_TestDroppedLabels = y_test.copy()
y_TestDroppedLabels.drop(labelsToDrop, axis = "columns", inplace = True)

pipeline = createPipeline(AdaBoostClassifier())

pipeline.fit(X_train, y_TrainDroppedLabels)
y_pred = pipeline.predict(X_test)

print("AdaBoost")
print(classification_report(y_TestDroppedLabels, y_pred, target_names = y_TestDroppedLabels.columns))

AdaBoost
                        precision    recall  f1-score   support

               related       0.80      0.96      0.88      5001
               request       0.78      0.52      0.62      1093
           aid_related       0.76      0.58      0.66      2700
          medical_help       0.58      0.24      0.34       532
      medical_products       0.63      0.32      0.42       345
     search_and_rescue       0.56      0.16      0.25       165
              military       0.53      0.24      0.33       197
                 water       0.76      0.67      0.71       408
                  food       0.81      0.66      0.73       723
               shelter       0.74      0.55      0.63       590
              clothing       0.71      0.36      0.48        95
                 money       0.52      0.30      0.38       138
        missing_people       0.58      0.19      0.29        74
              refugees       0.59      0.24      0.34       223
                 death       0

In [8]:
pipeline = createPipeline(LinearSVC())

pipeline.fit(X_train, y_TrainDroppedLabels)
y_pred = pipeline.predict(X_test)

print("LinearSVC with dropped Labels")
print(classification_report(y_TestDroppedLabels, y_pred, target_names = y_TestDroppedLabels.columns))

NameError: name 'y_TrainDroppedLabels' is not defined

### 9. Export your model as a pickle file

In [28]:
# The last used pipeline is the best performing one
pickle.dump(pipeline, open("model.pickle",'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [27]:
def toLabels(prediction):
    return y_TestDroppedLabels.columns[prediction[0] == 1]

print(
    toLabels(
        pipeline.predict(["there was an earthquake please help"]
)))

Index(['related', 'request', 'aid_related', 'other_aid', 'weather_related',
       'earthquake', 'direct_report'],
      dtype='object')
